# US Regional Sales Data Analysis and Prediction

## Project Overview

This project focuses on analyzing and predicting sales performance across different regions in the US, based on a comprehensive dataset containing information on various sales channels such as In-Store, Online, Distributor, and Wholesale. With 7,991 rows and 16 columns, the dataset provides insights into sales transactions, customer interactions, and product performance, enabling detailed analysis of sales patterns and trends. The goal of this project is to uncover meaningful insights from the data and build predictive models to forecast prices and optimize sales strategies.

## Source

This dataset is available on Kaggle in the following link:

> https://www.kaggle.com/datasets/talhabu/us-regional-sales-data/data

## Data Dictionary

The dataset includes detailed information on sales orders, customer interactions, and pricing. Below is a breakdown of the columns in the dataset:

- **OrderNumber**: A unique identifier for each order.
- **Sales Channel**: The channel through which the sale was made (In-Store, Online, Distributor, Wholesale).
- **WarehouseCode**: Code representing the warehouse involved in the order.
- **ProcuredDate**: Date when the products were procured.
- **OrderDate**: Date when the order was placed.
- **ShipDate**: Date when the order was shipped.
- **DeliveryDate**: Date when the order was delivered.
- **SalesTeamID**: Identifier for the sales team involved.
- **CustomerID**: Identifier for the customer.
- **StoreID**: Identifier for the store.
- **ProductID**: Identifier for the product.
- **Order Quantity**: Quantity of products ordered.
- **Discount Applied**: Applied discount for the order.
- **Unit Cost**: Cost of a single unit of the product.
- **Unit Price**: Price at which the product was sold.

This dataset serves as a valuable resource for analyzing sales trends, identifying high-performing products, assessing the effectiveness of different sales channels, and optimizing pricing strategies across various regions.

## Problem Statement

### Data Modelling and Machine Learning Ideas (Price Prediction)

1. **XGBoost Regression**: Use an XGBoost regression model to predict the price, incorporating multiple features and their interactions.

These visualizations and models will provide valuable insights into sales patterns and help optimize pricing strategies to improve overall sales performance.




### Load Libraries

In [1]:
# General
import pandas as pd
import numpy as np
import os
import warnings
import pickle

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Model and Evaluation Metrics
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Model Optimization
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

### Settings

In [2]:
# Warning
warnings.filterwarnings("ignore")
# Path
data_path = "../data"
model_path = "../models"
csv_path = os.path.join(data_path, "US_Regional_Sales_Data_Cleaned.csv")

### Load Data

In [3]:
df = pd.read_csv(csv_path)

In [4]:
# Check Data
df.head()

,Sales Channel,Order Quantity,Discount Applied,Unit Cost,Unit Price,DeliveryDays
0,In-Store,5,0.075,1001.18,1963.1,5
1,Online,3,0.075,3348.66,3939.6,0
2,Distributor,1,0.050,781.22,1775.5,0
3,Wholesale,8,0.075,1464.69,2324.9,150
4,Distributor,8,0.100,1476.14,1822.4,10


### Preproseccing

- Encode the categorical features
- Separate Input and output features for model traning
- Split data into training and testing set so that model can be trained on training set and test the performance of the metrics with test data.
- Scale the input data to standardize it as all numeric data are not in same scale.

In [5]:
# Use one hot encoding to encode the categorical feature Sales Channel as it contains 4 categories of data
df = pd.get_dummies(df, drop_first= True, dtype="int")
# Sanity check
df.head()

,Order Quantity,Discount Applied,Unit Cost,Unit Price,DeliveryDays,Sales Channel_In-Store,Sales Channel_Online,Sales Channel_Wholesale
0,5,0.075,1001.18,1963.1,5,1,0,0
1,3,0.075,3348.66,3939.6,0,0,1,0
2,1,0.050,781.22,1775.5,0,0,0,0
3,8,0.075,1464.69,2324.9,150,0,0,1
4,8,0.100,1476.14,1822.4,10,0,0,0


In [6]:
# Separate the input and output features for train the model
X = df.drop("Unit Price", axis= 1)
y = df["Unit Price"]

In [7]:
# Split the data to training and testing purpose
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

In [8]:
# Standardize the data by scaling
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

### Model Training and Evaluation

- Train the model with training dataset so that it can learn the pattern of the data.
- Evaluate the performance of the trained model with testing data using different evaluation metrics

In [24]:
# Function to train the model and print evaluation metrics
def train_evaluate(model):
    # Train the data
    model.fit(X_train, y_train)

    # Make prediction on training and testing data for evaluation
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Print Evaluation Metrics
    print("=" * 60)
    print("EVALUATION METRICS FOR TRAINING DATA")
    print("=" * 60)
    print(f"Accuracy: {r2_score(y_train, y_train_pred)}")
    print(f"Mean Absolute Error(MAE): {mean_absolute_error(y_train, y_train_pred): 0.2f}")
    print(f"Mean Squared Error(MAE): {mean_squared_error(y_train, y_train_pred):0.2f}")
    print(f"Root Mean Squared Error(RMAE): {np.sqrt(mean_squared_error(y_train, y_train_pred)):0.2f}")
    print("=" * 60)
    print("EVALUATION METRICS FOR TESTING DATA")
    print("=" * 60)
    print(f"Accuracy: {r2_score(y_test, y_test_pred)}")
    print(f"Mean Absolute Error(MAE): {mean_absolute_error(y_test, y_test_pred): 0.2f}")
    print(f"Mean Squared Error(MAE): {mean_squared_error(y_test, y_test_pred):0.2f}")
    print(f"Root Mean Squared Error(RMAE): {np.sqrt(mean_squared_error(y_test, y_test_pred)):0.2f}")

In [28]:
# Try SVM Regressor model
svr = SVR()
train_evaluate(svr)

EVALUATION METRICS FOR TRAINING DATA
Accuracy: 0.6480893215667118
Mean Absolute Error(MAE):  642.38
Mean Squared Error(MAE): 980214.30
Root Mean Squared Error(RMAE): 990.06
EVALUATION METRICS FOR TESTING DATA
Accuracy: 0.6436295574727884
Mean Absolute Error(MAE):  646.34
Mean Squared Error(MAE): 1016480.90
Root Mean Squared Error(RMAE): 1008.21


### Key Findings

- The **accuracy** values are very similar for both the training and testing data, which indicates that the model is generalizing well. It isn’t overfitting (which would result in much higher accuracy on training data than testing data), but the overall accuracy is somewhat moderate. A **64%** accuracy indicates that the model is able to correctly predict about **64%** of the time. Depending on the context of the dataset (for example, whether it's a very complex dataset), this might be acceptable or might suggest that the model can still be improved.
- The **Mean Absolute Error(MAE)** represents the average absolute difference between predicted and actual values. The values here suggest that, on average, the model’s predictions are off by approximately **642-646** units. Since the MAE for both training and testing is very close, the model is not overfitting or underfitting too much, but the magnitude of the error indicates that predictions are consistently off by a relatively large amount.
- The **Mean Squared Error(MSE)** gives more weight to larger errors due to squaring the differences, which means it’s more sensitive to outliers than MAE. The **large MSE** values suggest that there are some predictions where the model is making significantly large errors. These could potentially be **outliers** or cases where the model struggles to predict accurately.
- The **Root Mean Squared Error(RMSE)** is the square root of MSE and represents the average error magnitude. This gives a value in the same units as the target variable, making it easier to interpret. Here, the RMSE values of **~990 and ~1008** indicate that on average, the model is making prediction errors of around **990-1008** units. Again, the fact that the training and testing RMSE are close indicates that the model's performance is consistent across both sets, but the absolute error is still quite high.

### Model Optimization

- Use cross validation to check the score by 5 separate split
- Tune the different hyperparameter of the model with GridSearchCV to the best score with best hyperparameter set for the model. 

In [29]:
# Define KFold with 5 splits
kf = KFold(n_splits= 5)

# Define the SVM Regressor model
svr_cv = SVR(C=0.1, epsilon=0.5, kernel="linear")

# Find cross validation score for each split
cv_scores = cross_val_score(svr_cv, X, y, cv = kf)

# Print the mean  score
print(f"Cross Validattion Score: {np.mean(cv_scores): 0.2f}")

Cross Validattion Score:  0.88


In [31]:
#  Function to tune the hyperparameter
def tune_hyperparameter(model, param_dict):
    # Define GridSearchCV
    gscv = GridSearchCV(
        model,
        param_grid= param_dict,
        cv = 5,
        verbose= 1,
        scoring= "r2"
    )

    # Train model the different hyperparameter
    gscv.fit(X, y)

    # Print best score
    print(f"Best Score: {gscv.best_score_: 0.2f}")

    # Return best hyperparameter set
    best_params = gscv.best_params_
    return best_params

In [33]:
# Define hyperparameter dictionary for XGBoostRegressor
param_dict = {
    "C": [0.1],
    "epsilon": [0.5, 0.6],
    "kernel": ["linear"]
}

# Define SVM Regressor
svr_ht = SVR()

# Hyperpermeter tuning to get best hyperparameters
best_params = tune_hyperparameter(svr_ht, param_dict)
print(best_params)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best Score:  0.88
{'C': 0.1, 'epsilon': 0.6, 'kernel': 'linear'}


In [34]:
# Train the model with best huperparameters
model = SVR(**best_params)
train_evaluate(model)

EVALUATION METRICS FOR TRAINING DATA
Accuracy: 0.8831461255420643
Mean Absolute Error(MAE):  390.29
Mean Squared Error(MAE): 325485.55
Root Mean Squared Error(RMAE): 570.51
EVALUATION METRICS FOR TESTING DATA
Accuracy: 0.8755349155964015
Mean Absolute Error(MAE):  403.20
Mean Squared Error(MAE): 355013.67
Root Mean Squared Error(RMAE): 595.83


### Conclution

- **Accuracy Improvement**: The accuracy increased to around **88%**, a substantial improvement. This suggests that after hyperparameter tuning, the model is much better at making accurate predictions, indicating it has now captured more useful patterns in the data.
- **Error Reduction**:
  - **Mean Absolute Error(MAE)**: The MAE represents the average magnitude of errors. A reduction from **~642 to ~390** shows that the model's predictions are, on average, **250** units closer to the actual values compared to before tuning. This indicates a significant improvement in how closely the model is predicting sales or outcomes.
  - **Mean Squared Error(MSE)**: The MSE gives more weight to larger errors, so the fact that it dropped so significantly **(~67% decrease for training and ~65% for testing)** indicates that the model is now making fewer large errors compared to before tuning.
  - **Root Mean Squared Error(RMSE)**: The RMSE represents the average error magnitude in the same units as the target variable. A reduction from around **990-1008** to **~570-595** shows that the model's predictions have become much more precise, with errors decreasing by around **400** units on average.

In [35]:
#  Save the model
svm_path = os.path.join(model_path, "svm_reg_rsp.pkl")
with open(svm_path, "wb") as svm_file:
    pickle.dump(model, svm_file)